# 1. Import the Preprocessed Data

In [1]:
import pickle
import requests
import io
import numpy as np
import pandas as pd

pickle_file = 'https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_3/datasets/pickles/preprocessed_faces_data.pkl'
#data = pickle.load(io.BytesIO(requests.get(pickle_file).content))
response = requests.get(pickle_file)
data = pd.read_pickle(io.BytesIO(response.content)) 
data.keys()


dict_keys(['X_train', 'X_test', 'y_train_userid', 'y_train_pose', 'y_train_expression', 'y_train_eyes', 'y_test_userid', 'y_test_pose', 'y_test_expression', 'y_test_eyes'])

In [2]:
X_train = data['X_train']
X_test = data['X_test']

y_train_userid = data['y_train_userid']
y_train_pose = data['y_train_pose']
y_train_expression = data['y_train_expression']
y_train_eyes = data['y_train_eyes']

y_test_userid = data['y_test_userid']
y_test_pose = data['y_test_pose']
y_test_expression = data['y_test_expression']
y_test_eyes = data['y_test_eyes']

# 2. Build the Model

In [3]:
from tensorflow.keras import layers, models, Model
# First we build the input layer
input_layer = layers.Input(shape=(60, 64, 1), name='input_layer')

# Shared layers (common across all tasks)
# The second layer should be a Conv2D layer built off the input_layer
conv1 = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)

# The third layer should be a MaxPooling2D layer built off the second layer
maxpool1 = layers.MaxPooling2D((2, 2))(conv1)

# The fourth layer should be a Conv2D layer built off the third layer
conv2 = layers.Conv2D(64, (3, 3), activation='relu')(maxpool1)

# The fifth layer should be a MaxPooling2D layer built off the fourth layer
maxpool2 = layers.MaxPooling2D((2, 2))(conv2)

# The sixth layer should be a Conv2D layer built off the fifth layer
conv3 = layers.Conv2D(64, (3, 3), activation='relu')(maxpool2)

# The seventh layer should be a Flatten layer built off the sixth layer
flatten = layers.Flatten()(conv3)

# Lastly, build one dense layer before branching to the different y branches
dense_shared = layers.Dense(64, activation='relu')(flatten)


In [4]:
# Build the branches for each of the y variables
# Include a dense hidden layer in each along with the output layer.
# I need to remember to include the correct number of nodes for the output

# userid
userid_dense = layers.Dense(64, activation='relu')(dense_shared)
userid_output = layers.Dense(len(y_train_userid.columns),
                             activation='sigmoid',
                             name='userid_output')(userid_dense)

# pose
pose_dense = layers.Dense(64, activation='relu')(dense_shared)
pose_output = layers.Dense(len(y_train_pose.columns),
                           activation='softmax',
                             name='pose_output')(pose_dense)

# expression
expression_dense = layers.Dense(64, activation='relu')(dense_shared)
expression_output = layers.Dense(len(y_train_expression.columns),
                                 activation='softmax',
                             name='expression_output')(expression_dense)

# eyes
eyes_dense = layers.Dense(64, activation='relu')(dense_shared)
eyes_output = layers.Dense(len(y_train_eyes.columns),
                           activation='sigmoid',
                             name='eyes_output')(eyes_dense)

In [5]:
# Build the model
model = Model(inputs=input_layer, outputs=[
    userid_output,
    pose_output,
    expression_output,
    eyes_output
])

# Compile the model
model.compile(optimizer='adam',
              loss={'userid_output': 'categorical_crossentropy',
                    'pose_output': 'categorical_crossentropy',
                    'expression_output': 'categorical_crossentropy',
                    'eyes_output': 'binary_crossentropy'},
              metrics={'userid_output': 'accuracy',
                       'pose_output': 'accuracy',
                       'expression_output': 'accuracy',
                       'eyes_output': 'accuracy'})

In [6]:
# Train the model with the training data
model.fit(
    X_train,
    {
        'userid_output': y_train_userid,
        'pose_output': y_train_pose,
        'expression_output': y_train_expression,
        'eyes_output': y_train_eyes
    },
    epochs=10,  # I can adjust the number of epochs based on needs
    batch_size=32,  # I can adjust the batch size based on available memory
    validation_split=0.2  # I can specify the validation split if I have a separate validation set
)

Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - expression_output_accuracy: 0.2490 - eyes_output_accuracy: 0.5377 - loss: 6.4242 - pose_output_accuracy: 0.2863 - userid_output_accuracy: 0.0682 - val_expression_output_accuracy: 0.2628 - val_eyes_output_accuracy: 0.7265 - val_loss: 6.0452 - val_pose_output_accuracy: 0.3504 - val_userid_output_accuracy: 0.1752
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - expression_output_accuracy: 0.2768 - eyes_output_accuracy: 0.6834 - loss: 5.5463 - pose_output_accuracy: 0.4206 - userid_output_accuracy: 0.2754 - val_expression_output_accuracy: 0.2500 - val_eyes_output_accuracy: 0.7821 - val_loss: 4.8364 - val_pose_output_accuracy: 0.4936 - val_userid_output_accuracy: 0.4509
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - expression_output_accuracy: 0.3163 - eyes_output_accuracy: 0.7690 - loss: 4.4386 - pose_output_accuracy: 0.5186 - userid_output_accuracy: 0.4949 - val_expression_output_accuracy: 0.2799 - val_eyes_output_accuracy: 0.7

In [11]:
# Evaluate the model using the test data
results = model.evaluate(np.array(X_test), {
        'userid_output': y_test_userid,
        'pose_output': y_test_pose,
        'expression_output': y_test_expression,
        'eyes_output': y_test_eyes
    })

# Print the accuracy for each category
pred_categories = ['userid', 'pose', 'expression', 'eyes']
for i, cat in enumerate(pred_categories):
    print(f"{cat} accuracy: {results[i+1]}")



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - expression_output_accuracy: 0.1580 - eyes_output_accuracy: 0.7981 - loss: 3.7437 - pose_output_accuracy: 0.7549 - userid_output_accuracy: 0.9202 
userid accuracy: 0.1666666716337204
pose accuracy: 0.8461538553237915
expression accuracy: 0.7307692170143127
eyes accuracy: 0.9038461446762085
